# Data to Policy Spring 2021

Weston Grewe and Angela Morrison

University of Colorado Denver

Math 7594 Integer Programming

Instructor: Dr. Steffen Borgwardt

## Summary
A college degree is becoming a necessary requirement of entering the middle class. A college education is also an excellent way to lift people out of poverty. Some high schools have high immediate college enrollments while others have low immediate college enrollments. For any one high school, it is nearly impossible to determine which factors contribute significantly to college enrollment. Often, it is a blend of many factors such as class size, proportion of low income students, teacher pay, number of AP classes offered, and many other factors. 

In this project, we create an interpretable optimal decision tree to understand which factors make the greatest impact. We will use data from Massachusetts' public schools in 2017 which can be found on Kaggle.

In [1]:
import numpy as np
import pandas as pd

In [2]:
raw_data = pd.read_csv('MA_Public_Schools_2017.csv')

The dataset contains 302 fields for 1861 schools. This includes elementary, middle, and high schools as well as schools that serve many grade levels. We will begin by selecting only schools which serve senior high school students. A school which does not serve seniors cannot have immediate college enrollment. We will then select only fields which would be beneficial to this analysis. For example, the number of AP classes taken is relevant while the school's principal is not.

In [3]:
slice1 = raw_data[raw_data['12_Enrollment'] > 1]
slice1

School Code                                        School Name  \
0           10505                                      Abington High   
8           50505                                        Agawam High   
16          70505                                      Amesbury High   
17          70515                    Amesbury Innovation High School   
23          90505                                       Andover High   
...           ...                                                ...   
1851     39010900  Massachusetts Virtual Academy at Greenfield Co...   
1854     35010505         Paulo Freire Social Justice Charter School   
1855     35080505  Phoenix Academy Public Charter High School Spr...   
1856     35060505     Pioneer Charter School of Science II (PCSS-II)   
1858     39020900  TEC Connections Academy Commonwealth Virtual S...   

         School Type   Function            Contact Name  \
0      Public School  Principal    Teresa Sullivan-Cruz   
8      Public School  Principal          Thomas Schnepp   
16     Public School  Principal     Elizabeth McAndrews   
17     Public School  Principal            Eryn Maguire   
23     Public School  Principal           Philip Conrad   
...              ...        ...                     ...   
1851   Public School  Principal             Greg Runyan   
1854  Charter School  Principal          Melissa Mirhej   
1855  Charter School  Principal  Jacqueline Adam-Taylor   
1856  Charter School  Principal            Vahit Sevinc   
1858   Public School  Principal           Adam Goldberg   

                     Address 1     Address 2          Town State   Zip  ...  \
0           201 Gliniewicz Way           NaN      Abington    MA  2351  ...   
8            760 Cooper Street           NaN        Agawam    MA  1001  ...   
16           5 Highland Street           NaN      Amesbury    MA  1913  ...   
17            71 Friend Street           NaN      Amesbury    MA  1913  ...   
23           80 Shawsheen Road           NaN       Andover    MA  1810  ...   
...                        ...           ...           ...   ...   ...  ...   
1851              278 Main St.      Ste. 205    Greenfield    MA  1301  ...   
1854  161 Lower Westfield Road  P O Box 1009       Holyoke    MA  1041  ...   
1855         65 Lincoln Street           NaN   Springfield    MA  1105  ...   
1856            97 Main Street           NaN        Saugus    MA  1906  ...   
1858         141 Mansion Drive           NaN  East Walpole    MA  2032  ...   

     MCAS_10thGrade_English_Incl. in SGP(#)  \
0                                     111.0   
8                                     263.0   
16                                    133.0   
17                                      NaN   
23                                    310.0   
...                                     ...   
1851                                   20.0   
1854                                   48.0   
1855                                    3.0   
1856                                   30.0   
1858                                   28.0   

     Accountability and Assistance Level  \
0                                Level 1   
8                                Level 2   
16                               Level 2   
17                     Insufficient data   
23                               Level 2   
...                                  ...   
1851                             Level 3   
1854                   Insufficient data   
1855                   Insufficient data   
1856                   Insufficient data   
1858                             Level 3   

              Accountability and Assistance Description  \
0                           Meeting gap narrowing goals   
8                       Not meeting gap narrowing goals   
16                      Not meeting gap narrowing goals   
17                                                  NaN   
23                      Not meeting gap narrowing goals   
...                                

Now, we must choose which columns to include in our analysis. It would also be interesting to study mutable and immutable factors in two different analyses to understand what changes. For instance, some factors may be most determining, e.g. poverty/wealth, but schools have no control over these factors. For a decision, schools can only be concerned with mutable factors, e.g. teacher pay, number of AP classes. 

Factors (in order of Col #)
- School type (Public/Charter)
- ZIP 
- District/District Code
- Total Enrollment
- First Lang Not English
- English Lang Learner
- Disability
- High Need
- Economically Disadvantaged
- Race Makeup
- Average Class Size
- Average Salary
- Average Expenditure per Pupil
- % Graduated
- % Dropped Out
- AP Test takers
- Number of Tests Taken
- AP Score
- Average SAT Math
- Average SAT Reading
- Average SAT Writing
- 10th Grade MCAS (If used, filter for 10th graders)
- Accountability Metrics

Replace "%" symbols in column names to avoid possible errors in future column name calling.

In [4]:
slice1.columns = slice1.columns.str.replace('%', 'Percent');

Take only columns with information we care about.

In [5]:
>>> important_cols = slice1[['School Name', 'School Type', 'TOTAL_Enrollment', 'First Language Not English', 'Students With Disabilities','Percent Students With Disabilities', 'High Needs', 'Percent High Needs', 'Economically Disadvantaged','Percent Economically Disadvantaged', 'Percent African American', 'Percent Asian', 'Percent Hispanic', 'Percent White','Percent Native American', 'Percent Native Hawaiian, Pacific Islander', 'Percent Multi-Race, Non-Hispanic','Percent Males', 'Percent Females', 'Average Class Size', 'Percent Graduated', 'Percent Non-Grad Completers','Percent GED', 'Percent Dropped Out', 'High School Graduates (#)', 'Attending Coll./Univ. (#)', 'Percent Attending College','Percent Private Two-Year', 'Percent Private Four-Year', 'Percent Public Two-Year', 'Percent Public Four-Year', 'Percent MA Community College','Percent MA State University','Percent UMass', 'AP_Test Takers', 'AP_Tests Taken', 'AP_One Test', 'AP_Two Tests', 'AP_Three Tests','AP_Four Tests', 'AP_Five or More Tests','AP_Score=1', 'AP_Score=2', 'AP_Score=3', 'AP_Score=4', 'AP_Score=5','Percent AP_Score 1-2', 'Percent AP_Score 3-5', 'SAT_Tests Taken', 'Average SAT_Reading', 'Average SAT_Writing', 'Average SAT_Math']]

Get current shape of new dataframe and remove rows with any missing data and get shape of new dataset

In [6]:
print(important_cols.shape)
important_cols.isnull().sum().sum()
clean_imp_cols = important_cols.dropna()
print(clean_imp_cols.shape)

(393, 52)
(292, 52)


Print names of columns in cleaned dataset

In [7]:
clean_imp_cols.columns.tolist()

['School Name',
 'School Type',
 'TOTAL_Enrollment',
 'First Language Not English',
 'Students With Disabilities',
 'Percent Students With Disabilities',
 'High Needs',
 'Percent High Needs',
 'Economically Disadvantaged',
 'Percent Economically Disadvantaged',
 'Percent African American',
 'Percent Asian',
 'Percent Hispanic',
 'Percent White',
 'Percent Native American',
 'Percent Native Hawaiian, Pacific Islander',
 'Percent Multi-Race, Non-Hispanic',
 'Percent Males',
 'Percent Females',
 'Average Class Size',
 'Percent Graduated',
 'Percent Non-Grad Completers',
 'Percent GED',
 'Percent Dropped Out',
 'High School Graduates (#)',
 'Attending Coll./Univ. (#)',
 'Percent Attending College',
 'Percent Private Two-Year',
 'Percent Private Four-Year',
 'Percent Public Two-Year',
 'Percent Public Four-Year',
 'Percent MA Community College',
 'Percent MA State University',
 'Percent UMass',
 'AP_Test Takers',
 'AP_Tests Taken',
 'AP_One Test',
 'AP_Two Tests',
 'AP_Three Tests',
 'AP_Fo

Columns that need to be converted into binary variables:
* School Type
* Total Enrollment
* Percent Student with Disabilities
* Percent High Needs
* Percent Economically Disadvantaged
* Percent African American
* Percent Asian
* Percent Hispanic
* Percent White
* Percent Native American
* Percent Native Hawaiian. Pacific Islander
* Percent Multi-Race, Non-Hispanic
* Percent Males
* Percent Females
* Average Class Size
* Percent Graduated
* Percent Non-Grad Completers
* Percent GED
* Percent Dropped Out
* High School Graduates (#)
* Attending Coll./Univ. (#)
* Percent Attending College
* Percent Private Two-Year
* Percent Private Four-Year
* Percent Public Two-Year
* Percent Public Four-Year
* Percent MA Community College
* Percent MA State University
* Percent UMass
* AP_Test Takers
* AP_Tests Taken
* AP_One Test
* AP_Two Tests
* AP_Three Tests
* AP_Four Tests
* AP_Five or More Tests
* AP_Score=1
* AP_Score=2
* AP_Score=3
* AP_Score=4
* AP_Score=5
* Percent AP_Score 1-2
* Percent AP_Score 3-5
* SAT_Tests Taken
* Average SAT_Reading
* Average SAT_Reading
* Average SAT_Writing
* Average SAT_Math

Breakdown of non-binary columns:

In [8]:
print(clean_imp_cols['TOTAL_Enrollment'].mean())
print(clean_imp_cols['Percent Students With Disabilities'].mean())
print(clean_imp_cols['Percent High Needs'].mean())
print(clean_imp_cols['Percent Economically Disadvantaged'].mean())
print(clean_imp_cols['Percent African American'].mean())
print(clean_imp_cols['Percent Asian'].mean())
print(clean_imp_cols['Percent Hispanic'].mean())
print(clean_imp_cols['Percent White'].mean())
print(clean_imp_cols['Percent Native American'].mean())
print(clean_imp_cols['Percent Native Hawaiian, Pacific Islander'].mean())
print(clean_imp_cols['Percent Multi-Race, Non-Hispanic'].mean())
print('Males = ',clean_imp_cols['Percent Males'].mean())
print('Females =',clean_imp_cols['Percent Females'].mean())
print('Average Class Size=',clean_imp_cols['Average Class Size'].mean())
print(clean_imp_cols['Percent Graduated'].mean())
print(clean_imp_cols['Percent Non-Grad Completers'].mean())
print(clean_imp_cols['Percent GED'].mean())
print(clean_imp_cols['Percent Dropped Out'].mean())
print('High School Grads =',clean_imp_cols['High School Graduates (#)'].mean())
print('Attending college =',clean_imp_cols['Percent Attending College'].mean())
#clean_imp_cols['AP_Test Takers'].mean()
#clean_imp_cols['AP_Tests Taken'].mean()
print('AP Test 1 = ',clean_imp_cols['AP_One Test'].mean())
print('AP Test 2 = ',clean_imp_cols['AP_Two Tests'].mean())
print('AP Test 3 = ',clean_imp_cols['AP_Three Tests'].mean())
print('AP Test 4 = ',clean_imp_cols['AP_Four Tests'].mean())
print('AP Test 5+ = ',clean_imp_cols['AP_Five or More Tests'].mean())
print(clean_imp_cols['AP_Score=1'].mean())
print(clean_imp_cols['AP_Score=2'].mean())
print(clean_imp_cols['AP_Score=3'].mean())
print(clean_imp_cols['AP_Score=4'].mean())
print(clean_imp_cols['AP_Score=5'].mean())
print(clean_imp_cols['Percent AP_Score 1-2'].mean())
print(clean_imp_cols['Percent AP_Score 3-5'].mean())
print('SAT Taken = ',clean_imp_cols['SAT_Tests Taken'].mean())
print(clean_imp_cols['Average SAT_Reading'].mean())
print(clean_imp_cols['Average SAT_Writing'].mean())
print(clean_imp_cols['Average SAT_Math'].mean())

953.7020547945206
15.403424657534256
37.22191780821918
24.79041095890413
9.211986301369858
5.171232876712331
13.673630136986297
68.96643835616432
0.23150684931506857
0.09554794520547946
2.649657534246577
Males =  50.50205479452051
Females = 49.485958904109616
Average Class Size= 15.986643835616439
90.68561643835619
0.6750000000000003
0.7136986301369864
3.9328767123287665
High School Grads = 202.27397260273972
Attending college = 78.00821917808221
AP Test 1 =  85.08561643835617
AP Test 2 =  50.14383561643836
AP Test 3 =  25.215753424657535
AP Test 4 =  9.023972602739725
AP Test 5+ =  3.5753424657534247
42.87671232876713
61.96232876712329
77.61986301369863
72.4486301369863
61.03767123287671
41.660273972602766
58.34075342465751
SAT Taken =  161.63013698630138
502.6609589041096
489.51027397260276
513.9075342465753


Ideas for New Columns:
* 0-33%, 33-66% and 66-110% for emost percetnage columns. 
* Outcome column cut off is 64.6% (anything higher is a 1, otherwise 0)
* Average class size dived into 3rds also

In [9]:
def convert_to_thirds_perc(column,first_third,middle_third,last_third):
    for i in range(len(column)):
        if column[i] <= 33.0:
            first_third.append(1)
            middle_third.append(0)
            last_third.append(0)
        elif (column[i] > 33.0 and column[i] <=66.0):
            first_third.append(0)
            middle_third.append(1)
            last_third.append(0)
        else:
            first_third.append(0)
            middle_third.append(0)
            last_third.append(1)
            
    return first_third,middle_third,last_third

In [10]:
def convert_outcome_bin(column,bin_outcome):
    for i in range(len(column)):
        if column[i] < 64.6:
            bin_outcome.append(0)
        else:
            bin_outcome.append(1)
    
    return bin_outcome

In [11]:
#convert columns to arrays for functions
disab_array = np.array(clean_imp_cols['Percent Students With Disabilities'])
high_needs_array = np.array(clean_imp_cols['Percent High Needs'])
econ_dis_array = np.array(clean_imp_cols['Percent Economically Disadvantaged'])
african_array = np.array(clean_imp_cols['Percent African American'])
asian_array = np.array(clean_imp_cols['Percent Asian'])
hispanic_array = np.array(clean_imp_cols['Percent Hispanic'])
white_array = np.array(clean_imp_cols['Percent White'])
native_array = np.array(clean_imp_cols['Percent Native American'])
pacific_array = np.array(clean_imp_cols['Percent Native Hawaiian, Pacific Islander'])
multi_race_array = np.array(clean_imp_cols['Percent Multi-Race, Non-Hispanic'])

ap_test_taken_1_array = np.array(clean_imp_cols['AP_One Test'])
ap_test_taken_2_array = np.array(clean_imp_cols['AP_Two Tests'])
ap_test_taken_3_array = np.array(clean_imp_cols['AP_Three Tests'])
ap_test_taken_4_array = np.array(clean_imp_cols['AP_Four Tests'])
ap_test_taken_5_plus_array = np.array(clean_imp_cols['AP_Five or More Tests'])


#Create empty lists for conversion function
disab_33_below = []
disab_33_66 = []
disab_66_above = []

high_needs_33_below = []
high_needs_33_66 = []
high_needs_66_above = []

econ_dis_33_below = []
econ_dis_33_66 = []
econ_dis_66_above = []

african_33_below = []
african_33_66 = []
african_66_above = []

asian_33_below = []
asian_33_66 = []
asian_66_above = []

hispanic_33_below = []
hispanic_33_66 = []
hispanic_66_above = []

white_33_below = []
white_33_66 = []
white_66_above = []

native_33_below = []
native_33_66 = []
native_66_above = []

pacific_33_below = []
pacific_33_66 = []
pacific_66_above = []

multi_race_33_below = []
multi_race_33_66 = []
multi_race_66_above = []

ap_test_taken_1_33_below = []
ap_test_taken_1_33_66 = []
ap_test_taken_1_66_above = []

ap_test_taken_2_33_below = []
ap_test_taken_2_33_66 = []
ap_test_taken_2_66_above = []

ap_test_taken_3_33_below = []
ap_test_taken_3_33_66 = []
ap_test_taken_3_66_above = []

ap_test_taken_4_33_below = []
ap_test_taken_4_33_66 = []
ap_test_taken_4_66_above = []

ap_test_taken_5_plus_33_below = []
ap_test_taken_5_plus_33_66 = []
ap_test_taken_5_plus_66_above = []

#Calling conversion fucntion
disab_33_below,disab_33_66,disab_66_above = convert_to_thirds_perc(disab_array,disab_33_below,disab_33_66,disab_66_above);
high_needs_33_below,high_needs_33_66,high_needs_66_above = convert_to_thirds_perc(high_needs_array,high_needs_33_below,high_needs_33_66,high_needs_66_above);
econ_dis_33_below,econ_dis_33_66,econ_dis_66_above = convert_to_thirds_perc(econ_dis_array,econ_dis_33_below,econ_dis_33_66,econ_dis_66_above);
african_33_below,african_33_66,african_66_above = convert_to_thirds_perc(african_array,african_33_below,african_33_66,african_66_above);
asian_33_below,asian_33_66,asian_66_above = convert_to_thirds_perc(asian_array,asian_33_below,asian_33_66,asian_66_above);
hispanic_33_below,hispanic_33_66,hispanic_66_above = convert_to_thirds_perc(hispanic_array,hispanic_33_below,hispanic_33_66,hispanic_66_above);
white_33_below,white_33_66,white_66_above = convert_to_thirds_perc(white_array,white_33_below,white_33_66,white_66_above);
native_33_below,native_33_66,native_66_above = convert_to_thirds_perc(native_array,native_33_below,native_33_66,native_66_above);
pacific_33_below,pacific_33_66,pacific_66_above = convert_to_thirds_perc(pacific_array,pacific_33_below,pacific_33_66,pacific_66_above);
multi_race_33_below,multi_race_33_66,multi_race_66_above = convert_to_thirds_perc(multi_race_array,multi_race_33_below,multi_race_33_66,multi_race_66_above);


ap_test_taken_1_33_below,ap_test_taken_1_33_66,ap_test_taken_1_66_above = convert_to_thirds_perc(ap_test_taken_1_array,ap_test_taken_1_33_below,ap_test_taken_1_33_66,ap_test_taken_1_66_above);
ap_test_taken_2_33_below,ap_test_taken_2_33_66,ap_test_taken_2_66_above = convert_to_thirds_perc(ap_test_taken_2_array,ap_test_taken_2_33_below,ap_test_taken_2_33_66,ap_test_taken_2_66_above);
ap_test_taken_3_33_below,ap_test_taken_3_33_66,ap_test_taken_3_66_above = convert_to_thirds_perc(ap_test_taken_3_array,ap_test_taken_3_33_below,ap_test_taken_3_33_66,ap_test_taken_3_66_above);
ap_test_taken_4_33_below,ap_test_taken_4_33_66,ap_test_taken_4_66_above = convert_to_thirds_perc(ap_test_taken_4_array,ap_test_taken_4_33_below,ap_test_taken_4_33_66,ap_test_taken_4_66_above);
ap_test_taken_5_plus_33_below,ap_test_taken_5_plus_33_66,ap_test_taken_5_plus_66_above = convert_to_thirds_perc(ap_test_taken_5_plus_array,ap_test_taken_5_plus_33_below,ap_test_taken_5_plus_33_66,ap_test_taken_5_plus_66_above);

In [12]:
#convert college percentage to binary outcomes
outcome_array = np.array(clean_imp_cols['Percent Attending College'])
outcome_bin = []

outcome_bin = convert_outcome_bin(outcome_array,outcome_bin)

In [14]:
columns_to_add = [disab_33_below,disab_33_66,disab_66_above,high_needs_33_below,high_needs_33_66,\
                  high_needs_66_above,econ_dis_33_below,econ_dis_33_66,econ_dis_66_above,\
                  african_33_below,african_33_66,african_66_above,asian_33_below,asian_33_66,\
                  asian_66_above,hispanic_33_below,hispanic_33_66,hispanic_66_above,white_33_below,\
                  white_33_66,white_66_above,native_33_below,native_33_66,native_66_above,pacific_33_below,\
                  pacific_33_66,pacific_66_above,multi_race_33_below,multi_race_33_66,multi_race_66_above,\
                  ap_test_taken_1_33_below,ap_test_taken_1_33_66,ap_test_taken_1_66_above,ap_test_taken_2_33_below,\
                  ap_test_taken_2_33_66,ap_test_taken_2_66_above,ap_test_taken_3_33_below,ap_test_taken_3_33_66,\
                  ap_test_taken_3_66_above,ap_test_taken_4_33_below,ap_test_taken_4_33_66,ap_test_taken_4_66_above,\
                  ap_test_taken_5_plus_33_below,ap_test_taken_5_plus_33_66,ap_test_taken_5_plus_66_above]

new_col_names = ['Disability Percent 0-33','Disability Percent 33-66','Disability Percent 66-100',\
                 'High Needs Percent 0-33','High Needs Percent 33-66','High Needs Percent 66-100',\
                 'Econ Disadvantage Percent 0-33','Econ Disadvantage Percent 33-66','Econ Disadvantage 66-100',\
                 'African American 0-33','African American Percent 33-66','African American Percent 66-100',\
                 'Asian Percent 0-33','Asian Percent 33-66','Asian Percent 66-100','Hispanic Percent 0-33',\
                 'Hispanic Percent 33-66','Hispanic Percent 66-100','White Percent 0-33','White Percent 33-66',\
                 'White Percent 66-100','Native Percent 0-33','Native Percent 33-66','Native Percent 66-100',\
                 'Pacific Percent 0-33','Pacific Percent 33-66','Pacific Percent 66-100','Multi-Race Percent 0-33',\
                 'Multi-Race Percent 33-66','Multi-Race Percent 66-100','1 AP Test Percent 0-33',\
                 '1 AP Test Percent 33-66','1 AP Test Percent 66-100','2 AP Test Percent 0-33',\
                 '2 AP Test Percent 33-66','2 AP Test Percent 66-100','3 AP Test Percent 0-33',\
                 '3 AP Test Percent 33-66','3 AP Test Percent 66-100','4 AP Test Percent 0-33',\
                 '4 AP Test Percent 33-66','4 AP Test Percent 66-100','5+ AP Test Percent 0-33',\
                 '5+ AP Test Percent 33-66','5+ AP Test Percent 66-100']

# clean_imp_cols = clean_imp_cols.reindex(columns=clean_imp_cols.columns.tolist() + new_col_names)

for i in range(len(new_col_names)):
    clean_imp_cols[new_col_names[i]] = pd.DataFrame(columns_to_add[i], index=clean_imp_cols.index) 

<ipython-input-14-220187edd6a3>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_imp_cols[new_col_names[i]] = pd.DataFrame(columns_to_add[i], index=clean_imp_cols.index)


In [16]:
clean_imp_cols.columns.tolist()

['School Name',
 'School Type',
 'TOTAL_Enrollment',
 'First Language Not English',
 'Students With Disabilities',
 'Percent Students With Disabilities',
 'High Needs',
 'Percent High Needs',
 'Economically Disadvantaged',
 'Percent Economically Disadvantaged',
 'Percent African American',
 'Percent Asian',
 'Percent Hispanic',
 'Percent White',
 'Percent Native American',
 'Percent Native Hawaiian, Pacific Islander',
 'Percent Multi-Race, Non-Hispanic',
 'Percent Males',
 'Percent Females',
 'Average Class Size',
 'Percent Graduated',
 'Percent Non-Grad Completers',
 'Percent GED',
 'Percent Dropped Out',
 'High School Graduates (#)',
 'Attending Coll./Univ. (#)',
 'Percent Attending College',
 'Percent Private Two-Year',
 'Percent Private Four-Year',
 'Percent Public Two-Year',
 'Percent Public Four-Year',
 'Percent MA Community College',
 'Percent MA State University',
 'Percent UMass',
 'AP_Test Takers',
 'AP_Tests Taken',
 'AP_One Test',
 'AP_Two Tests',
 'AP_Three Tests',
 'AP_Fo

Save cleaned data to csv file to be used in AMPL

In [ ]:
clean_imp_cols.to_csv('clean_school_data.csv', index=False)

In [ ]:
clean_imp_cols.to_csv('clean_school_data.csv', index=False)